# ESERCIZIO 2 - PIPELINE DATABRICKS

## OBIETTIVO
Creare una pipeline personalizzata con i dataset di bike sharing, trasformazioni, join e schedulazione con timezone asiatico.

# STEP 0: SETUP DATABRICKS
- [ ] Accedere a **Databricks Workspace**
- [ ] Andare su **Workspace** > **Create** > **Notebook**
- [ ] Nome: `Esercizio_2_Pipeline`
- [ ] Language: **Python**
- [ ] Cluster: selezionare cluster attivo

# STEP 1: PREPARAZIONE DATI

## Bronze Layer
```python
# Lettura dati grezzi
df_day = spark.read.csv("/databricks-datasets/bikeSharing/data-001/day.csv", header=True, inferSchema=True)
df_hour = spark.read.csv("/databricks-datasets/bikeSharing/data-001/hour.csv", header=True, inferSchema=True)
```

## Silver Layer - Trasformazioni
```python
# PRIMA TABELLA: Classificare i giorni in base al numero di utenti registrati
def classifica_giorno(utenti_registrati):
    if utenti_registrati > 5000:
        return "Alto"
    elif utenti_registrati > 2000:
        return "Medio"
    else:
        return "Basso"

from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
classifica_udf = udf(classifica_giorno, StringType())
df_day_clean = df_day.withColumn("livello_utilizzo", classifica_udf(col("registered")))

# SECONDA TABELLA: Aggiungere un campo calcolato per l'ora della giornata
from pyspark.sql.functions import concat, lpad
df_hour_clean = df_hour.withColumn("data_ora", concat(col("dteday"), lpad(col("hr"), 2, '0')))
```

## Gold Layer - Join finale
```python
# Creare un dataset finale che unisca i dati giornalieri con quelli orari
df_finale = df_day_clean.join(df_hour_clean, "dteday", "inner")
```

# STEP 3: SALVATAGGIO SCHEMI
```sql
CREATE DATABASE IF NOT EXISTS bronze;
CREATE DATABASE IF NOT EXISTS silver;  
CREATE DATABASE IF NOT EXISTS gold;
```

```python
# Salvare nelle tabelle
df_day.write.mode("overwrite").saveAsTable("bronze.bike_sharing_day_raw")
df_hour.write.mode("overwrite").saveAsTable("bronze.bike_sharing_hour_raw")
df_day_clean.write.mode("overwrite").saveAsTable("silver.bike_sharing_day_clean")
df_hour_clean.write.mode("overwrite").saveAsTable("silver.bike_sharing_hour_clean")
df_finale.write.mode("overwrite").saveAsTable("gold.bike_sharing_final")
```

# STEP 3.1: CREAZIONE PIPELINE
- [ ] Andare in **Delta Live Tables** > **Create Pipeline
- [ ] Nome: `Bike_Sharing_Pipeline`
- [ ] Source code: selezionare il notebook `Esercizio_2_Pipeline`
- [ ] Destination: selezionare il catalogo e lo schema appropriati
- [ ] Cluster mode: selezionare **Fixed size** o **Enhanced autoscaling**
- [ ] Pipeline mode: **Triggered**
- [ ] Cliccare su **Create**

# STEP 4: CREAZIONE JOB
- [ ] Andare in **Workflows** > **Create Job**
- [ ] Nome: `Pipeline_Esercizio_2`
- [ ] Task type: **Notebook**
- [ ] Source: selezionare il notebook `Esercizio_2_Pipeline`
- [ ] Cluster: selezionare cluster esistente

# STEP 5: SCHEDULAZIONE ASIATICA
- [ ] Schedule: **Scheduled**
- [ ] Cron: `0 9 * * *` (ogni giorno ore 9:00)
- [ ] Timezone: **Asia/Tokyo**

# STEP 6: NOTIFICHE
- [ ] Notifications > On Success: inserire email
- [ ] Notifications > On Failure: inserire email
- [ ] **Run Now** per testare

In [ ]:
%sql
-- create catalog, schemas and volume
CREATE CATALOG IF NOT EXISTS test_catalog;
CREATE SCHEMA IF NOT EXISTS test_catalog.test_schema;
CREATE VOLUME IF NOT EXISTS test_catalog.test_schema.test_volume;
CREATE SCHEMA IF NOT EXISTS test_catalog.test_bronze_schema;
CREATE SCHEMA IF NOT EXISTS test_catalog.test_silver_schema;
CREATE SCHEMA IF NOT EXISTS test_catalog.test_gold_schema;